In [5]:
!pip install -U transformers torch langchain peft bitsandbytes faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.8/410.8 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00


In [7]:
!pip install torch torchvision

In [8]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=882435b292d459b23a8c91a4ac0e6b8de33333dd45fe3e285d32ad87eba20361
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# 모델 경로 설정
base_model_path = "qwen2.5-7b-finetuned/final_model"  #  원본 모델
lora_model_path = "qwen2.5-7b-finetuned/final_model"  #  LoRA 가중치 경로

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(base_model_path)

# 원본 모델 로드 (VRAM 최적화 적용)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    device_map="auto",
    torch_dtype=torch.float16,  
    load_in_4bit=True,  # VRAM 부족하면 4-bit 양자화
    llm_int8_enable_fp32_cpu_offload=True  # CPU로 일부 연산 오프로드
)

# LoRA 가중치 적용
model = PeftModel.from_pretrained(
    base_model,
    lora_model_path,
    config_name="adapter_config.json"  # 어댑터 설정 명시
)

print(" LoRA 적용 완료! (원본 모델 + 어댑터)")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


✅ LoRA 적용 완료! (원본 모델 + 어댑터)


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import pandas as pd
import re


#  데이터 로드 (박물관 데이터)
data_path = "./data/merged_museum_data.csv"  # 본인 데이터 경로 확인!
df = pd.read_csv(data_path)

#  문장 분할 함수 (간단한 마침표 기준 + 공백 제거)
def split_sentences(text):
    # 마침표, 느낌표, 물음표 기준으로 문장 분리 (뒤에 공백도 포함)
    sentences = re.split(r'(?<=[.?!])\s+', text.strip())
    return [s for s in sentences if s]  # 빈 문자열 제거

#  Title + Description을 문장 단위로 쪼개서 문서 생성
documents = []
for _, row in df.iterrows():
    title = row["Title"]
    description = row["Description"]
    sentences = split_sentences(description)

    for sentence in sentences:
        documents.append({
            "text": f"{title} - {sentence}",
            "metadata": {"title": title}
        })

#  임베딩 모델
embedding_model = HuggingFaceEmbeddings(model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS")

#  FAISS 벡터스토어 생성
vectorstore = FAISS.from_texts(
    [doc["text"] for doc in documents],
    embedding_model,
    metadatas=[doc["metadata"] for doc in documents]
)

#  저장
vectorstore.save_local("faiss_index")
print("✅ 문장 단위 벡터DB 생성 완료!")


<ipython-input-4-04b149338c2e>:31: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/467M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ 문장 단위 벡터DB 생성 완료!


In [ ]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools import WikipediaQueryRun

# Wikipedia API 래퍼 인스턴스 생성
wiki_api = WikipediaAPIWrapper(lang="ko")  # 한국어 위키 검색 설정

#  Wikipedia 검색 도구 설정
wikipedia_tool = WikipediaQueryRun(api_wrapper=wiki_api)

print(" Wikipedia 검색 설정 완료!")


✅ Wikipedia 검색 설정 완료!


In [ ]:
# 유사도 임계값 설정 (0.7 이상인 문서만 채택)
SIMILARITY_THRESHOLD = 0.7

def filter_similar_docs(docs):
    filtered_docs = [doc for doc in docs if doc["similarity"] >= SIMILARITY_THRESHOLD]
    return filtered_docs if filtered_docs else "🔹 해당 정보는 없습니다."

# 질문 실행 함수 (유사도 체크)
def ask_question(query):
    results = retriever_museum.get_relevant_documents(query)
    filtered_results = filter_similar_docs(results)

    if filtered_results == "🔹 해당 정보는 없습니다.":
        return "🔹 해당 정보는 없습니다. 위키피디아 검색을 진행합니다."

    return qa_chain({"question": query, "context": filtered_results})


In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

# FAISS 벡터 DB 불러오기 (보안 옵션 추가)
embedding_model = HuggingFaceEmbeddings(model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS")
vectorstore = FAISS.load_local(
    "/content/faiss_index",
    embedding_model,
    allow_dangerous_deserialization=True  # 보안 설정 추가
)

retriever_museum = vectorstore.as_retriever(search_kwargs={"k": 3})  # 상위 3개 검색

print(" FAISS 벡터DB 로드 완료!")

# LLM 변환 (HuggingFacePipeline) => LangChain에서 사용 가능하게 변경
llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto"
)

llm = HuggingFacePipeline(pipeline=llm_pipeline)  # LangChain과 호환되는 형태로 변환

#  대화 기록 저장 (Memory 적용)
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question")

# `ConversationalRetrievalChain` 생성 (QA Chain)
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,  # LoRA 적용된 모델 사용
    retriever=retriever_museum,  # FAISS 기반 문서 검색
    memory=memory,
)

print(" RAG + LoRA LLM 설정 완료!")


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForCausalLM', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCaus

✅ FAISS 벡터DB 로드 완료!
✅ RAG + LoRA LLM 설정 완료!


In [ ]:
def ask_question(query):
    response = qa_chain({"question": query})
    return response["answer"]  # 답변 부분만 반환

In [ ]:
# 테스트 실행
user_input = "김수철이 누구야?"
response = ask_question(user_input)
print("🔹 AI 응답:", response)


🔹 AI 응답: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

개자원화전 - 중국 청나라 초엽의 화가 왕개왕시왕얼 3형제가 편찬한 화보로 개자원이라는 말은 이 화보의 편찬을 후원했던 청대 유명한 문인이자 대부호였던 이어16111680의 별장 이름에서 따온 것이다 전체 4집 중 신수19794는 제 2집으로 심인백의 의뢰로 왕개왕시왕얼 형제가 편집한 매란국죽보이다 내용은 총목과 본문으로 구성되어 있다 제첨에 개자원화전 상해 천보서국 발행이라 기재되어 있고 권두에는 1888년 14에 쓴 서문이 실려있다

허성 묘지명 백자 철화 묘지명 - 허성 1548년1612년의 묘지명이다 완전한 한 벌이 아니지만 벼슬 이력 등을 통하여 주인공을 밝혔다 허성은 허봉 허균 허난설헌과 친 동기간이다 사헌부 대사헌인 지봉 이수광이 글을 지었고 손자가 하였다고 기록되어 있다 지봉집에도 수록되어 있지 않다

호적표 - 1897년광무 1 충청남도 홍산군 남면 마동리 제6통 제3호 유생 박준달의 호적표이다 홍산군수 이재화의 관인이 찍혀 있다조선시대에 작성된 가족사항을 적어 신고하는 서류인 호구단자와 달리 갑오개혁 이후 새롭게 정해진 양식에 적혀 있는 것이 특징이다 적혀 있는 내용은 대략 다음과 같다 먼저 6통의 통수 곽선국의 이름이 적혀 있다 호주 박준달은 나이 48세 본관은 밀양 직업은 농사이다 사조 아버지할아버지증조외조를 살펴보면 아버지는 학생 정진 생부는 학생 건진 할아버지는 학생 기회 증조부는 학생 갑환 외조부는 학생 송업룡이다 처 문씨의 본관은 남평이며 아들 하나 딸 하나를 두고 있어 현재 살아 있는 사람은 네 명이다 집은 초가 세 칸을 소유하고 있는 것으로 적혀 있다

Question: 김수철이 누구야?
Helpful Answer: 김수철은